# Contribution analysis on the foreground system

In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bw.projects.set_current('contr-analysis-fg') # a database where I have ecoinvent
bw.databases

Databases dictionary with 1 object(s):
	fg-ca-exmpl

# Create a foreground system

Three activities A, B, and C. Each of them emit CO2. B is an input to A. Both activtiy A and activity C have an input of C (nested system).

In [4]:
bw2_db = {('fg-ca-exmpl', 'Activity A'): {'name': 'Activity A',
  'unit': 'kilogram',
  'type': 'process',
  'exchanges': [{'input': ('fg-ca-exmpl', 'Activity A'),
    'amount': 1.0,
    'unit': 'kilogram',
    'type': 'production',
    'uncertainty type': 0},
   {'input': ('fg-ca-exmpl', 'Activity B'),
    'amount': 0.5,
    'unit': 'kilogram',
    'type': 'technosphere',
    'uncertainty type': 0},
   {'input': ('fg-ca-exmpl', 'Activity C'), # input of C to A
    'amount': 0.5,
    'unit': 'kilogram',
    'type': 'technosphere',
    'uncertainty type': 0},
   {'input': ('fg-ca-exmpl', 'Carbon Dioxide'),
    'amount': 1.0,
    'unit': 'kilogram',
    'type': 'biosphere',
    'uncertainty type': 0}]},
 ('fg-ca-exmpl', 'Activity B'): {'name': 'Activity B',
  'unit': 'kilogram',
  'type': 'process',
  'exchanges': [{'input': ('fg-ca-exmpl', 'Activity B'),
    'amount': 10.0,
    'unit': 'kilogram',
    'type': 'production',
    'uncertainty type': 0},
   {'input': ('fg-ca-exmpl', 'Activity C'), # input of C to B
    'amount': 10.0,
    'unit': 'kilogram',
    'type': 'technosphere',
    'uncertainty type': 0},
   {'input': ('fg-ca-exmpl', 'Carbon Dioxide'),
    'amount': 12.5,
    'unit': 'kilogram',
    'type': 'biosphere',
    'uncertainty type': 0}]},
 ('fg-ca-exmpl', 'Activity C'): {'name': 'Activity C',
  'unit': 'kilogram',
  'type': 'process',
  'exchanges': [{'input': ('fg-ca-exmpl', 'Activity C'),
    'amount': 15.0,
    'unit': 'kilogram',
    'type': 'production',
    'uncertainty type': 0},
   {'input': ('fg-ca-exmpl', 'Carbon Dioxide'),
    'amount': 15.0,
    'unit': 'kilogram',
    'type': 'biosphere',
    'uncertainty type': 0}]},
 ('fg-ca-exmpl', 'Carbon Dioxide'): {'name': 'Carbon Dioxide', # a foreground exchange just to keep things simple
  'unit': 'kilogram',
  'type': 'biosphere'}}

In [5]:
del(bw.databases['fg-ca-exmpl'])
fgca_db = bw.Database('fg-ca-exmpl')
fgca_db.write(bw2_db)

Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/09/2023 10:11:48
  Finished: 12/09/2023 10:11:48
  Total time elapsed: 00:00:00
  CPU %: 78.00
  Memory %: 1.10


In [13]:
# check it worked
activities = [('fg-ca-exmpl', act['code']) for act in bw.Database('fg-ca-exmpl')]
activities

[('fg-ca-exmpl', 'Activity A'),
 ('fg-ca-exmpl', 'Activity B'),
 ('fg-ca-exmpl', 'Activity C'),
 ('fg-ca-exmpl', 'Carbon Dioxide')]

In [34]:
for act in bw.Database('fg-ca-exmpl'):
    print(act.as_dict())
    for exc in list(act.exchanges()):
        print(exc)
    print('---')

{'name': 'Activity A', 'unit': 'kilogram', 'type': 'process', 'database': 'fg-ca-exmpl', 'code': 'Activity A'}
Exchange: 1.0 kilogram 'Activity A' (kilogram, None, None) to 'Activity A' (kilogram, None, None)>
Exchange: 0.5 kilogram 'Activity B' (kilogram, None, None) to 'Activity A' (kilogram, None, None)>
Exchange: 0.5 kilogram 'Activity C' (kilogram, None, None) to 'Activity A' (kilogram, None, None)>
Exchange: 1.0 kilogram 'Carbon Dioxide' (kilogram, None, None) to 'Activity A' (kilogram, None, None)>
---
{'name': 'Activity B', 'unit': 'kilogram', 'type': 'process', 'database': 'fg-ca-exmpl', 'code': 'Activity B'}
Exchange: 10.0 kilogram 'Activity B' (kilogram, None, None) to 'Activity B' (kilogram, None, None)>
Exchange: 10.0 kilogram 'Activity C' (kilogram, None, None) to 'Activity B' (kilogram, None, None)>
Exchange: 12.5 kilogram 'Carbon Dioxide' (kilogram, None, None) to 'Activity B' (kilogram, None, None)>
---
{'name': 'Carbon Dioxide', 'unit': 'kilogram', 'type': 'biosphere'

In [36]:
# we create a basic LCIA method to include LCIA phase

myLCIAdata = [[('fg-ca-exmpl', 'Carbon Dioxide'), 1.0]]
method_key = ('dummy GWI method','none','none')
my_method = bw.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register() 
my_method.write(myLCIAdata)
my_method.load()

[[('fg-ca-exmpl', 'Carbon Dioxide'), 1.0]]

In [37]:
fu_amount = 100 # the functional unit for this analysis

In [38]:
# Calculate total score per FU
mymethod = ('dummy GWI method','none','none')
act = fgca_db.get('Activity A')
functional_unit = {act: fu_amount} # the functional unit refers to the output of A
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
tot_score = lca.score
print("total score is", tot_score)

total score is 262.5


This is the total score of the system and we remember it for later

# Contribution analysis

## Super simple

Make a dictionary of activities and the "right" amount, iterate throught those.

In [70]:
ca_dict = {'Activity A' : 1 * fu_amount, 
           'Activity B' : 0.5 * fu_amount, # the total amount of B used in the system
           'Activity C' : (0.5 + 0.5 * 10/10) * fu_amount} # the total amount of C used in the system

In [71]:
#Use this if you are using the nested version of the system
#{'Activity A' : 1 * fu_amount, 
# 'Activity B' : 0.5 * fu_amount , # the total amount of B used in the system
# 'Activity C' : (0.5 + 0.5 * 10/10) * fu_amount } # the total amount of C used in the system (0.5 by A and 0.5 by B)

In [72]:
ca_results = []

for i in ca_dict:
    
    act = fgca_db.get(i)
    functional_unit = {act: ca_dict[i]}
    lca = bw.LCA(functional_unit, mymethod)
    lca.lci()
    lca.lcia()
    ca_results.append(lca.score)
    

ca_simple = pd.DataFrame(dict(zip([i for i in ca_dict], ca_results)), index = ['GWI']).T
ca_simple

,GWI
Activity A,262.5
Activity B,112.5
Activity C,100.0


The problem is, calculating the right amount might be difficult depending on how "nested" is this system. 
In this case it is nested because both A and B require C. This gets more complicated the more nested the system is and the approach is going to give errors very easily on more complex systems...

## A better approach based on LCA matrix algebra

We use the scaling vector that tells how much each ativity is needed to perform the FU.

Note that the scaling vector is **specific to the FU** so for a differnet FU (even a different amount) one needs to calcualte a new scaling vecotr and redo the contribution analysis.

In [42]:
# Initiate the inventory to get the scaling factors. These will be the scaling factor for the FU here chosen
fu_ca = bw.Database('fg-ca-exmpl').get('Activity A') 
lca = bw.LCA({fu_ca: fu_amount },) # contribution analysis for this FU only (and only that!)
lca.lci()

In [44]:
# list of all foreground activities excluding biosphere ones
acts = [act['code'] for act in bw.Database('fg-ca-exmpl') if act['type'] == 'process']

In [46]:
# dictionary of scaling factors from the supply array
scaling = {}
for act in acts:
    index = lca.activity_dict[('fg-ca-exmpl', act)] # index is to find the activities in the tech matrix
    scaling[act] = lca.supply_array[index] # this is the amount of activity used, we need it for contribution analysis

scaling

{'Activity C': 6.666666666666667, 'Activity B': 5.0, 'Activity A': 100.0}

In [47]:
# Dictionary of reference flows, because they are not always normalized!
refflows = {}
for act in acts:
    for exc in list(bw.Database('fg-ca-exmpl').get(act).exchanges()):
        if exc['type'] == 'production':
            refflows[act] = (exc['amount'])
refflows

{'Activity C': 15.0, 'Activity B': 10.0, 'Activity A': 1.0}

Refernece flows **might not be unitary** (e.g., the output exchange of B is 10 kg) so we need to account for this. 
We normalise the scaling factors using the reference flows

In [51]:
# getting the scaling factors normalized by reference flow
scaling_norm = dict(zip(acts,[scaling[i]*refflows[i] for i in acts]))
scaling_norm

{'Activity C': 100.0, 'Activity B': 50.0, 'Activity A': 100.0}

In [52]:
# summary showing the normalised scaling factors that are are scaling factors times reference flow
factors = pd.DataFrame([scaling, refflows, scaling_norm], index = ['Scaling','Reference flow', 'Scaling normalized']).T
factors

,Scaling,Reference flow,Scaling normalized
Activity C,6.666667,15.0,100.0
Activity B,5.000000,10.0,50.0
Activity A,100.000000,1.0,100.0


Calculate the life cycle impact of **one unit of each activity** (unitaty impact) and save it. 

This is the cumulative impact considering all the activities linked to it over the life cycle.

In [55]:
#defining some functions that simply do LCA 
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = bw.LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

def getLCAresults(acts, mymethod):
    
    all_activities = []
    results = []
    for a in acts:
        act = bw.Database('fg-ca-exmpl').get(a)
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict

In [56]:
unitary_results = []
unitary_results_all_acts = getLCAresults(acts,mymethod) # total impact per unitary value of each activity
unitary_results.append(unitary_results_all_acts)
unitary_results # the impact of one unit of each process
my_output = pd.DataFrame(unitary_results)
my_output.index = ['GWI']
my_output.T.sort_index().head()

,GWI
Activity A,2.625
Activity B,2.250
Activity C,1.000


Here below we multiply the unitary impact by the normalised scaling vector and we obtain the absolute impact of each activity considered base don how much this activity is required to provide the cuntional unit. **This is the contribution analysis** in absolute terms, then further below we can calcualte is as a fraction of the total impact.

In [62]:
ca_scores_cumulative = pd.DataFrame(my_output.T.sort_index().values * pd.DataFrame(factors['Scaling normalized']).sort_index().values, 
             index = my_output.T.sort_index().index, columns = my_output.index)
ca_scores_cumulative

,GWI
Activity A,262.5
Activity B,112.5
Activity C,100.0


The sum of B + C does not gives A  **in this particular case** because: 

- A has also some impacts on its own (direct emission and input form background) 
- The contribution from B includes impacts that derive from C (nested system)

If the system was build more "flat" or with activities "in parallel" (a number of activities that all contribute to the same top activity but don't require input of each other) then the sum will give the total impact.


Also note that this is the same as calculated before witht he simple approach... but here we will not risk to make mistakes

In [73]:
#need to use some rounding to check this...using round at the 10th digit after the comma
sign = 10
[round(i, sign) for i in ca_scores_cumulative['GWI'].values] == [round(i,sign) for i in ca_simple['GWI'].values]

True

In [74]:
ca_scores_cumulative / tot_score # to calcualte in percentage

,GWI
Activity A,1.000000
Activity B,0.428571
Activity C,0.380952


This shuold be intended as the cumulative contribution, so B contributes for 42% of impacts and C for 38% (but B includes some of the impact of C).

#### Note: remember that this contribution analysis is *only* for the functional unit specified when initializing the LCI, to do it for a different functional unit a different set of scaling factors needs to be calculated.

# Additional Contribution analysis

One can also look at the **direct** inputs to the FU, considering only the activities just one "step" or "layer" upstream the activity where the FU is chosen or, in other words, looking at the exchange inputs associated with a specific exchange output. This is as Simapro would do it to calculate contribution analysis _(in fact Simapro does many different contirbution analysies including the one presented before...)_

In [75]:
fu_direct_inputs = {}

for exc in list(bw.Database('fg-ca-exmpl').get('Activity A').exchanges()):
    if exc['type'] != 'biosphere':
        fu_direct_inputs[exc['input'][1]] = exc['amount'] * fu_amount  # this is the key line

fu_direct_inputs 

{'Activity A': 100.0, 'Activity B': 50.0, 'Activity C': 50.0}

In [77]:
factors = pd.DataFrame([scaling, refflows, scaling_norm, fu_direct_inputs], index = ['Scaling','Reference flow', 'Scaling normalized', 'Direct inputs per FU']).T
factors

,Scaling,Reference flow,Scaling normalized,Direct inputs per FU
Activity C,6.666667,15.0,100.0,50.0
Activity B,5.000000,10.0,50.0,50.0
Activity A,100.000000,1.0,100.0,100.0


In [78]:
ca_scores_direct = pd.DataFrame(my_output.T.sort_index().values * pd.DataFrame(factors['Direct inputs per FU']).sort_index().values, 
             index = my_output.T.sort_index().index, 
             columns = my_output.index)

ca_scores_direct

,GWI
Activity A,262.5
Activity B,112.5
Activity C,50.0


A problem here is that the first row shows the total impact. We can fix it manually.

In [79]:
total_contr = ca_scores_direct.loc['Activity A','GWI']

In [80]:
# a long code to sum all results but the first row...
upstream_contr = ca_scores_direct[~ca_scores_direct.index.isin(['Activity A'])]['GWI'].sum() 

In [81]:
direct_contr = total_contr - upstream_contr

In [82]:
ca_scores_direct_SP = ca_scores_direct.copy()
ca_scores_direct_SP.at['Activity A', 'GWI'] = direct_contr
ca_scores_direct_SP

,GWI
Activity A,100.0
Activity B,112.5
Activity C,50.0


This is how Simapro shows it. I haven't found a way to calculate this in one go using the matrices. Maybe there isn't.

In [83]:
ca_scores_direct_SP / tot_score # to calcualte in percentage

,GWI
Activity A,0.380952
Activity B,0.428571
Activity C,0.190476


B contributes 42 % while C contributes 19%. This is how much they contribute **direclty** to A which is the top activity, or, more precisely, the output exchange chosen as functional unit and to which the other input exchanges refer to. Here we can see that A contributes 38% to the total impact via direct emissions.

In [85]:
ca_scores_direct_SP['GWI'].sum() / tot_score

1.0

and the sum gives 100% of course.